Unsupervised Change Detection

A lot of remote sensing DL models have been trained on RGB images. This is not useful as I am planning to use the Tree Cover derived from a deep learning model and stabilized with a Kalman Filter Smoother. 

# K-means

Let's try to use K-means to detect changes in the tree cover. 

In [10]:
import os
import rasterio
from sklearn.cluster import KMeans
import numpy as np

def read_and_sort_files(directory):
    # Retrieve all files from the directory
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.tif')]
    # Sort files based on the date in the filename
    files.sort(key=lambda x: x.split('/')[-1].split('_')[0])  # Adjust splitting based on your filename structure
    return files

def compute_change_map(file1, file2):
    # Read the two raster files
    with rasterio.open(file1) as src1:
        image1 = src1.read(1)  # Read the first band

    with rasterio.open(file2) as src2:
        image2 = src2.read(1)  # Read the first band

    # Compute the difference image
    diff = image2 - image1

    #keep only the negative values
    diff = np.where(diff < 0, diff, 0)
    
    # Reshape diff for k-means
    reshaped_diff = diff.reshape(-1, 1)

    # Apply K-means clustering
    kmeans = KMeans(n_clusters=2, random_state=0).fit(reshaped_diff)
    labels = kmeans.labels_.reshape(diff.shape)

    # Assuming the cluster with the higher mean is the change
    change_map = labels if kmeans.cluster_centers_[1] > kmeans.cluster_centers_[0] else 1 - labels
    return change_map, src1.meta

def write_change_map(change_map, meta, output_folder, output_filename):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Update metadata for a single band output
    meta.update(count=1, dtype='uint8')

    output_path = os.path.join(output_folder, output_filename + '.tif')
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(change_map.astype('uint8'), 1)



In [11]:
import os 
from tqdm import tqdm 
folder_path = '/Users/arthurcalvi/Data/disturbances-ex/bbox_sampling/324_20141228_20231228_fr-BourgogneFrancheComte-HauteSaone_Lat47.88_Lon6.60'
folder_treecover = os.path.join(folder_path, 'smoothed_treecover')
files = read_and_sort_files(folder_treecover)
output_directory = os.path.join(folder_path, 'change_maps_kmeans')
for i in tqdm(range(len(files) - 1)):
    change_map, meta = compute_change_map(files[i], files[i+1])
    date = files[i+1].split('/')[-1].split('_')[0]  # Adjust based on your filename structure
    write_change_map(change_map, meta, output_directory, date)


100%|██████████| 38/38 [00:10<00:00,  3.61it/s]


In [ ]:

# Directory containing your raster files
input_directory = 'path_to_your_raster_files'


# Processing
files = read_and_sort_files(input_directory)
for i in range(len(files) - 1):
    change_map, meta = compute_change_map(files[i], files[i+1])
    date = files[i+1].split('/')[-1].split('_')[0]  # Adjust based on your filename structure
    write_change_map(change_map, meta, output_directory, date)

Looking in indexes: https://pypi.org/simple, https://pypi.dev-kayrros.ovh/simple/
  Using cached affine-2.4.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached numpy-1.26.4-cp310-cp310-macosx_10_9_x86_64.whl.metadata (61 kB)
  Using cached snuggs-1.4.7-py3-none-any.whl.metadata (3.4 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl.metadata (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Using cached threadpoolctl-3.4.0-py3-none-any.whl.metadata (13 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 36.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 41.4 MB/s eta 